# Phase 0.0: Environment Setup

Setup PyTorch, CUDA, and configure RTX A5000 as default GPU for training.

## Contents
1. Check system and GPU info
2. Install required packages
3. Configure default GPU (A5000)
4. Verify setup

## 1. System Information

In [ ]:
# Check system info
import platform
import sys

print("=" * 60)
print("System Information")
print("=" * 60)
print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"Processor: {platform.processor()}")

In [ ]:
# Check available GPUs
!nvidia-smi

In [ ]:
# List GPU details
!nvidia-smi --query-gpu=index,name,memory.total,memory.free,driver_version --format=csv

## 2. Install Required Packages

In [ ]:
# Core ML packages
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# Transformers and related
!pip install transformers>=4.40.0
!pip install datasets>=2.18.0
!pip install accelerate>=0.27.0
!pip install peft>=0.10.0
!pip install trl>=0.8.0
!pip install bitsandbytes>=0.43.0

In [ ]:
# Tokenizer training
!pip install sentencepiece>=0.2.0
!pip install tokenizers>=0.15.0

In [ ]:
# Memory optimization
!pip install deepspeed>=0.14.0
!pip install flash-attn --no-build-isolation

In [ ]:
# Evaluation and utilities
!pip install lm-eval>=0.4.0
!pip install wandb
!pip install tensorboard
!pip install tqdm
!pip install pandas
!pip install matplotlib
!pip install seaborn

In [ ]:
# Deployment
!pip install vllm>=0.4.0
!pip install autoawq>=0.2.0

## 3. Configure Default GPU (RTX A5000)

In [ ]:
import torch
import os

def find_a5000_gpu():
    """Find RTX A5000 GPU index"""
    if not torch.cuda.is_available():
        print("CUDA not available!")
        return None
    
    n_gpus = torch.cuda.device_count()
    print(f"Found {n_gpus} GPU(s):")
    
    a5000_idx = None
    
    for i in range(n_gpus):
        name = torch.cuda.get_device_name(i)
        props = torch.cuda.get_device_properties(i)
        memory_gb = props.total_memory / (1024**3)
        
        print(f"  GPU {i}: {name} ({memory_gb:.1f} GB)")
        
        # Check for A5000 (48GB or 24GB variant)
        if "A5000" in name or "RTX A5000" in name:
            a5000_idx = i
            print(f"    -> Found RTX A5000 at index {i}")
    
    return a5000_idx

a5000_idx = find_a5000_gpu()

In [ ]:
def set_default_gpu(gpu_idx=None):
    """Set default GPU for PyTorch"""
    
    if gpu_idx is None:
        # Try to find A5000, otherwise use GPU 0
        gpu_idx = find_a5000_gpu()
        if gpu_idx is None:
            gpu_idx = 0
            print(f"A5000 not found, using GPU {gpu_idx}")
    
    # Set CUDA_VISIBLE_DEVICES
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_idx)
    
    # Set default device
    torch.cuda.set_device(0)  # After CUDA_VISIBLE_DEVICES, it becomes index 0
    
    print(f"\nDefault GPU set to: {torch.cuda.get_device_name(0)}")
    print(f"CUDA_VISIBLE_DEVICES: {os.environ.get('CUDA_VISIBLE_DEVICES', 'not set')}")
    
    return gpu_idx

# Set A5000 as default (or specify index manually)
# If you have multiple GPUs and know the A5000 index, use: set_default_gpu(1)
DEFAULT_GPU = set_default_gpu(a5000_idx)

In [ ]:
# Save GPU config for other notebooks
import json

gpu_config = {
    "default_gpu_idx": DEFAULT_GPU,
    "gpu_name": torch.cuda.get_device_name(0),
    "total_memory_gb": torch.cuda.get_device_properties(0).total_memory / (1024**3),
    "cuda_version": torch.version.cuda,
    "pytorch_version": torch.__version__,
}

config_path = "../config/gpu_config.json"
os.makedirs("../config", exist_ok=True)

with open(config_path, "w") as f:
    json.dump(gpu_config, f, indent=2)

print(f"GPU config saved to {config_path}")
print(json.dumps(gpu_config, indent=2))

## 4. Create GPU Setup Helper

In [ ]:
# Create a helper module for other notebooks
helper_code = '''"""GPU Setup Helper for Korean MedGemma Training"""

import os
import json
import torch

def setup_gpu(config_path="../config/gpu_config.json"):
    """Load GPU config and set as default"""
    
    # Load config if exists
    if os.path.exists(config_path):
        with open(config_path, "r") as f:
            config = json.load(f)
        
        gpu_idx = config.get("default_gpu_idx", 0)
        os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_idx)
    
    # Verify CUDA
    if not torch.cuda.is_available():
        raise RuntimeError("CUDA not available!")
    
    device = torch.device("cuda:0")
    
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")
    
    return device

def get_memory_info():
    """Get current GPU memory usage"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / (1024**3)
        reserved = torch.cuda.memory_reserved() / (1024**3)
        total = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        
        return {
            "allocated_gb": allocated,
            "reserved_gb": reserved,
            "total_gb": total,
            "free_gb": total - reserved,
        }
    return None

def print_memory_usage():
    """Print current GPU memory usage"""
    info = get_memory_info()
    if info:
        print(f"GPU Memory: {info[\'allocated_gb\']:.2f} GB allocated, "
              f"{info[\'free_gb\']:.2f} GB free / {info[\'total_gb\']:.1f} GB total")

def clear_memory():
    """Clear GPU memory cache"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        print("GPU memory cache cleared")
'''

os.makedirs("../config", exist_ok=True)
with open("../config/gpu_utils.py", "w") as f:
    f.write(helper_code)

print("Created ../config/gpu_utils.py")

## 5. Verify Setup

In [ ]:
# Test imports
print("Testing imports...")

import torch
print(f"  torch: {torch.__version__}")

import transformers
print(f"  transformers: {transformers.__version__}")

import datasets
print(f"  datasets: {datasets.__version__}")

import peft
print(f"  peft: {peft.__version__}")

import accelerate
print(f"  accelerate: {accelerate.__version__}")

import sentencepiece
print(f"  sentencepiece: {sentencepiece.__version__}")

try:
    import bitsandbytes
    print(f"  bitsandbytes: {bitsandbytes.__version__}")
except:
    print("  bitsandbytes: not installed")

print("\nAll imports successful!")

In [ ]:
# Test CUDA operations
print("Testing CUDA operations...")

# Create tensor on GPU
x = torch.randn(1000, 1000, device="cuda")
y = torch.randn(1000, 1000, device="cuda")

# Matrix multiplication
z = torch.matmul(x, y)

print(f"  Matrix multiplication on GPU: {z.shape}")
print(f"  Result device: {z.device}")

# Clean up
del x, y, z
torch.cuda.empty_cache()

print("\nCUDA operations successful!")

In [ ]:
# Test HuggingFace Hub access
from huggingface_hub import HfApi

print("Testing HuggingFace Hub access...")
api = HfApi()

# Check if we can access datasets
try:
    info = api.dataset_info("sean0042/KorMedMCQA")
    print(f"  KorMedMCQA dataset: {info.id}")
    print(f"  Downloads: {info.downloads}")
except Exception as e:
    print(f"  Warning: Could not access HuggingFace Hub: {e}")

print("\nHuggingFace Hub access test complete!")

In [ ]:
# Final summary
print("=" * 60)
print("Environment Setup Complete!")
print("=" * 60)
print(f"\nGPU: {torch.cuda.get_device_name(0)}")
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")
print(f"CUDA: {torch.version.cuda}")
print(f"PyTorch: {torch.__version__}")
print(f"\nConfig saved to: ../config/gpu_config.json")
print(f"Helper module: ../config/gpu_utils.py")
print("\nReady for Phase 0.1: Dataset Research!")